In [67]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
from seastar.seastar.gmfs.doppler import *
from seastar.seastar.oscar.oscar import *

In [71]:
for_file="D:\data\SEASTAR\SEASTARex\Data\Metasensing\OSCAR\Brest_Sample_June_28_2022\SAR_CPLX_20220517T093239_13.5G_VV_33_pres_1_fdc_auto.sar_INF_SAR_CPLX_20220517T093239_13.5G_VV_34_pres_1_fdc_auto.sar.nc"
aft_file="D:\data\SEASTAR\SEASTARex\Data\Metasensing\OSCAR\Brest_Sample_June_28_2022\SAR_CPLX_20220517T093239_13.5G_VV_77_pres_1_fdc_auto.sar_INF_SAR_CPLX_20220517T093239_13.5G_VV_78_pres_1_fdc_auto.sar.nc"
mid_file="D:\data\SEASTAR\SEASTARex\Data\Metasensing\OSCAR\Brest_Sample_June_28_2022\SAR_CPLX_20220517T093239_13.5G_VV_00_pres_1_fdc_auto.sar.nc"
dsm = xr.open_dataset(mid_file, mask_and_scale=True)
dsf = xr.open_dataset(for_file, mask_and_scale=True)
dsa = xr.open_dataset(aft_file, mask_and_scale=True)

In [72]:
dsf=compute_SLC_Master_Slave(dsf)
dsa=compute_SLC_Master_Slave(dsa)
dsf=compute_multilooking_Master_Slave(dsf, window=7)
dsa=compute_multilooking_Master_Slave(dsa, window=7)
dsa=compute_incidence_angle(dsa)
dsf=compute_incidence_angle(dsf)
#dsf=compute_antenna_azimuth_direction(dsf, 'fore')
#dsa=compute_antenna_azimuth_direction(dsa, 'aft')

In [73]:
m,n=dsf.OrbTimeImage.shape
head=np.interp(np.reshape(dsf.OrbTimeImage,(1,m*n)), dsf.GPSTime, dsf.OrbitHeading)
head=np.reshape(head,(dsf.OrbTimeImage.shape))
dsf['OrbitHeadingImage'] = xr.DataArray.copy(dsf.OrbTimeImage,data=head)
dsf.OrbitHeadingImage.attrs['long_name']='Heading (degrees from North) of the airfraft for each pixel in the image'
dsf.OrbitHeadingImage.attrs['units']='[degrees]'
dsf['AntennaAzimuthImage'] = np.mod( dsf.OrbitHeadingImage - 45, 360 )

In [61]:
dsf.OrbitHeadingImage.attrs['long_name']='Heading (degrees from North) of the airfraft for each pixel in the image'
dsf.OrbitHeadingImage.attrs['units']='[degrees]'

In [63]:
compute_antenna_azimuth_direction(dsf, options='fore')

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (array([[nan, nan, nan, ..., nan, nan,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), ['CrossRange', 'GroundRange']) to Variable.